In [1]:
%pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import ortools
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# from google.colab import files

In [65]:
def read_file(path):
    print(path)

    file = open(path, 'r')
    for line in file:
        flight_numbers, worker_numbers = line.split()
        break
        
    df = pd.read_csv(path, sep=' ', header=None, skiprows=1)
    df.head()
    costs=[]
    for item in df.to_numpy():
      item_costs=[]
      for task in item:
        if task ==1:
          item_costs.append(0)
        else:
          item_costs.append(100)
      costs.append(item_costs)
    num_workers = int(worker_numbers)
    num_tasks = int(flight_numbers)
    return costs,num_workers,num_tasks,path

In [92]:
def optimize(costs,num_workers,num_tasks,path):
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        print('solver init error')
    # x[j, i] is an array of 0-1 variables, which will be 1
    # if worker j is assigned to flight i.
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i, j] = solver.IntVar(0, 1, '')
    # Each worker is assigned to at most 1 task.
    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) == 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) >= 1)
    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs[j][i] * x[i, j])
    solver.Minimize(solver.Sum(objective_terms))
    status = solver.Solve()
    result=[]
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        for i in range(num_workers):
            for j in range(num_tasks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    result.append((j,i,-1 if costs[j][i]==100 else 1))
                    pass
        res_df=pd.DataFrame(result,columns=['Flight','Worker','Assigned'])
        # display(res_df)
        assign_res=[]
        for item in res_df.groupby('Flight').sum()['Assigned'].to_numpy():
          if item<0:
            assign_res.append(-1)
          else:
            assign_res.append(item)
        print(assign_res)
        # print(res_df.groupby('Flight').sum()['Cost'].to_numpy())
        # res_df.to_csv(f'{path}-result.csv')
        # files.download(f'{path}-result.csv')
    else:
        print('No solution found.')


In [93]:
for item in range(1,32):
  path =  str(item).zfill(2)
  # print(path)
  optimize(*read_file(path))

    

01
[2, 1, -1]
02
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
03
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
04
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1